# Tutorial

This notebook provides a basic tutorial on how to implement three basic tasks using the [ArDCA](https://github.com/pagnani/ArDCAData) package in [Julia](https://julialang.org/). We will assume we have a Multiple Sequence Alignment (MSA)in FASTA format. We aim at

1. Given a MSA, generate a sample
2. Given a MSA, predict contacts
3. Given a MSA, predict the mutational effect in all (ungapped) position of a given target sequence 


## Installing the software

1. Install [julia-1.5.3](https://julialang.org/downloads/)

2. Install the ipynb interface in julia, you should add the [IJulia](https://github.com/JuliaLang/IJulia.jl) package using the package manager. Again, from the package manager prompt do a
```
(@v1.5) pkg> add IJulia 
```
Exit the Package Manager (type `backspace` or `delete`) and from julia prompt do a 
```
julia> using IJulia
julia> notebook()
```
The ipynb file manager should open a page on your browser and from there you should navigate to this notebook.


## Load ArDCA package 

The following cell loads the package `ArDCA` on `nprocs` workers (here `nprocs=8`) (*Warning*: the first time it takes a while)


* The `mypkgdir` variable should be set to your `path/to/package` dir.

In this notebook we will use the PF00014 protein family available in `data/PF14/` folder of the package/


In [1]:
mypkgdir = normpath(joinpath(pwd(),".."))
cd(mypkgdir)
using Distributed
nprocs = 8
addprocs(nprocs) # choose the number of cores that you want to use (here nprocs=8)
@everywhere using Pkg
@everywhere Pkg.activate(".")
@everywhere using ArDCA

      From worker 3:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 2:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 6:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 8:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 5:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 4:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 9:	 Activating environment at `~/CODE/ArDCA/Project.toml`
      From worker 7:	 Activating environment at `~/CODE/ArDCA/Project.toml`


 Activating environment at `~/CODE/ArDCA/Project.toml`
┌ Info: Precompiling ArDCA [fc7e833f-a3b9-439d-ae82-c6ad11a3e54b]
└ @ Base loading.jl:1278


## Learn the autoregressive parameters

As a preliminary step, we learn the field and the coupling parameters $h,J$ from the MSA. To do so we use the `ardca` method that return the parameters (stored in `arnet` in the cell below), and the alignment in numerical format and other algorithms variables (stored in `arvar` in the cell below). The default autoregressive order is set to `:ENTROPIC`. We set the $L_2$ regularization to 0.02 for the $J$ and 0.001 for the $h$.

The keywork arguments for the `ardca` method are (with their default value):

* `epsconv::Real=1.0e-5` (convergenge parameter)

* `maxit::Int=1000` (maximum number of iteration - don't change)

* `verbose::Bool=true` (set to `false` to suppress printing on screen)

* `method::Symbol=:LD_LBFGS` (optimization method)

* `permorder::Union{Symbol,Vector{Ti}}=:ENTROPIC` (permutation order). Possible values are: `[:NATURAL, :ENTROPIC, :REV_ENTROPIC, :RANDOM]` or a custom permutation vector.




In [2]:
arnet,arvar=ardca("data/PF14/PF00014_mgap6.fasta.gz", verbose=false, lambdaJ=0.02,lambdaH=0.001);

removing duplicate sequences... done: 13600 -> 8871
M = 8871 N = 53 Meff = 2950.9255307616663 theta = 0.3308846126401137


## 1. Sequence Generation

We now generate `M` sequences using the `sample` method:

In [3]:
M = 1_000;
generated_alignment = sample(arnet,M);

The generated alignment has is a $L\times M$ matrix of integer where $L$ is the alignment's length, and $M$ the number of samples.

Interestingly, we for each sequence we can also compute the likelihood with the `sample_with_weights` method.

In [4]:
loglikelihood,generated_alignment = sample_with_weights(arnet,M);

## 2. Contact Prediction

We can compute the epistatic score for residue-residue contact prediction. To do so, we can use the `epistatic_score` method. The epistatic score is computed on any target sequence of the MSA. Empirically, it turns out the the final score does not depend much on the choice of the target sequence. 

The autput is contained in a `Vector` of `Tuple` ranked in descendic score order. Each `Tuple` contains $i,j,s_{ij}$ where $s_{ij}$ is the epistatic score of the residue pair $i,j$. The residue numbering is that of the MSA, and not of the unaligned full sequences.

In [5]:
target_sequence = 1
score=epistatic_score(arnet,arvar,target_sequence)

1378-element Array{Tuple{Int64,Int64,Float64},1}:
 (11, 35, 1.4170400334677726)
 (17, 41, 1.10593007716442)
 (21, 28, 1.0808850152980345)
 (45, 49, 0.7621114678424217)
 (6, 19, 0.6633330961068219)
 (17, 43, 0.5931715290477366)
 (18, 45, 0.5576410659908957)
 (13, 15, 0.5351640005192606)
 (10, 33, 0.5185564555706912)
 (24, 28, 0.48032213394096684)
 (19, 28, 0.4694920759570835)
 (37, 38, 0.42761260454811656)
 (44, 47, 0.41187795827863044)
 ⋮
 (12, 38, -0.12695136399636134)
 (37, 43, -0.1276334479959319)
 (7, 18, -0.12912649249480815)
 (35, 38, -0.13107173348549575)
 (28, 37, -0.1331229870092951)
 (20, 33, -0.13475007398011996)
 (28, 38, -0.13907534306887712)
 (15, 38, -0.13961353799510184)
 (11, 31, -0.14050857169554834)
 (11, 45, -0.14121866874682137)
 (33, 41, -0.14537808198988605)
 (11, 41, -0.15252388018123988)

## 3. Predicting mutational effects

For any reference sequence, we can easily predict the mutational effect for all single mutants. Of course we can extract this information only for the *non-gapped* residues of the target sequence. 

This is done with the `dms_single_site` method, which returns a `q×L` matrix `D` containing $\log(P(mut))/\log(P(wt))$ for all single
site mutants of the reference sequence `seqid` (i.e. the so-called wild type sequence), and `idxgap` a vector of indices of the residues of the reference sequence that contain gaps (i.e. the 21
amino-acid) for which the score has no sense and is set by convention to `+Inf`.

A negative value indicate a beneficial mutation, a value 0 indicate
the wild-type amino-acid.

In [6]:
target_sequence = 1
D,idxgap=dms_single_site(arnet,arvar,target_sequence)

([Inf 6.9360223404716805 … 6.937858289924648 Inf; Inf 0.0 … 0.0 Inf; … ; Inf 6.9360223404716805 … 6.937858289924648 Inf; Inf 2.90962217436406 … 3.4532928754048555 Inf], [18, 23])

In [7]:
D

21×53 Array{Float64,2}:
 Inf  6.93602  1.76391   -0.615337    …  -2.22543   -1.07414    6.93786  Inf
 Inf  0.0      4.25414    2.31167         1.15868    2.50552    0.0      Inf
 Inf  6.93602  1.41607    0.0            -1.50571    1.54949    6.93786  Inf
 Inf  6.93602  1.38093   -1.55619        -2.58418    0.0728249  6.93786  Inf
 Inf  6.93602  1.52179    0.392003        0.0       -0.113285   6.93786  Inf
 Inf  6.93602  2.43433    0.43931     …  -0.544059   1.61108    6.93786  Inf
 Inf  6.93602  2.08625    0.148112       -0.818548   0.0        6.93786  Inf
 Inf  6.93602  2.34401    0.685034       -0.613174  -0.242488   6.93786  Inf
 Inf  6.93602  1.32683   -0.0908006      -2.52321   -0.890128   6.93786  Inf
 Inf  6.93602  0.0       -2.45168        -1.44878   -0.628336   6.93786  Inf
 Inf  6.93602  1.96021   -0.760595    …  -0.492249   0.0808935  6.93786  Inf
 Inf  6.93602  1.02573    1.48681        -2.04058    0.461729   6.93786  Inf
 Inf  6.93602  4.07353    1.46396         1.25433   

In [8]:
idxgap

2-element Array{Int64,1}:
 18
 23